In [1]:
import json
import re
from typing import List, Callable, Dict, Set, Optional, Any

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
class TestLLM:
    def __init__(self):
        self.increment = 0

    def respond(self, messages) -> str:
        return r"""
<script type="application/json">
{
  "node_position": "Node 2 of 3",
  "channels": {
    "plot": {
      "messages": []
    },
    "characters": {
      "messages": []
    },
    "theme": {
      "messages": []
    },
    "style": {
      "messages": []
    }
  },
  "content": {
    "previous_node": "",
    "current_node": "and then I woke up",
    "next_node": ""
  }
}
</script>
"""
        

In [6]:
class ChatGptLLM:
    def __init__(self, client, model="gpt-4o-mini"):
        self.client = client
        self.model = model

    def respond(self, content: str) -> str:
        
        messages = [{"role": "user", "content": content}]
        
        # Make the API call using the injected completion function
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            # Extract the assistant's reply
            reply = response.choices[0].message.content
            return reply
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [39]:
class StoryAgent:
    def __init__(self, llm, node_position: int, total_nodes: int, prev_llm=None, next_llm=None, message_limit=10):
        self.llm = llm  # The agent's LLM object (assumed to have a respond method)
        self.prev_llm = prev_llm  # Reference to the previous agent
        self.next_llm = next_llm  # Reference to the next agent
        self.message_limit = message_limit  # Limit for stored messages
        self.node_position = node_position  # Position of the current node
        self.total_nodes = total_nodes  # Total number of nodes in the chain
        self.state_index = 0  # Tracks which "ping-pong" state we're using (0 or 1)
        
        # Two states to "ping-pong" between for channels and node content
        self.channels = [
            {  # State 0
                "plot": {"messages": []},
                "characters": {"messages": []},
                "theme": {"messages": []},
                "style": {"messages": []}
            },
            {  # State 1
                "plot": {"messages": []},
                "characters": {"messages": []},
                "theme": {"messages": []},
                "style": {"messages": []}
            }
        ]

        self.current_node = ["", ""]  # Two states for node content
    
    def get_channels(self):
        return self.channels[self.state_index]

    def get_current_state(self):
        return self.current_node[self.state_index]

    def strip_comments(self, response: str) -> str:
        """Strips single-line (//) and multi-line (/* */) comments from the response."""
        # Remove single-line comments
        response = re.sub(r"//.*?$", "", response, flags=re.MULTILINE)
        # Remove multi-line comments
        response = re.sub(r"/\*.*?\*/", "", response, flags=re.DOTALL)
        return response

    def extract_json(self, response: str) -> Any:
        """Extracts JSON object from within <script> tags or triple backticks (```json) in the response."""
        
        # Check for JSON inside <script> tags first
        json_match = re.search(r"<script[^>]*>(.*?)</script>", response, re.DOTALL)
        
        # If not found in <script> tags, check for JSON inside ```json blocks
        if not json_match:
            json_match = re.search(r"```json\s*(.*?)\s*```", response, re.DOTALL)
        
        # If any JSON content is found
        if json_match:
            json_str = json_match.group(1)

            # Strip comments from the JSON string
            json_str = self.strip_comments(json_str)

            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                print("Error decoding JSON.")
        
        return None

    def update_channels(self, new_data: Dict[str, Dict]) -> None:
        """Updates channels with new messages and removes old ones beyond the limit."""
        next_state_index = (self.state_index + 1) % 2
        next_channels = self.channels[next_state_index]  # Use the current state's channels
        
        for channel, updates in new_data.items():
            if channel not in next_channels:
                next_channels[channel] = {"messages": []}
            
            # Ensure you're extending the list inside the "messages" key
            next_channels[channel]["messages"].extend(updates.get("messages", []))
            
            # Keep only the most recent messages according to the message limit
            next_channels[channel]["messages"] = next_channels[channel]["messages"][-self.message_limit:]


    def construct_prompt(self) -> str:
        """Constructs the prompt to send to the agent, including neighboring node data."""
        prev_node = self.prev_llm.current_node[self.state_index] if self.prev_llm else ""
        next_node = self.next_llm.current_node[self.state_index] if self.next_llm else ""

        # Convert channels dictionary to JSON string and append it to the prompt
        payload = {
            "node_position": f"Node {self.node_position + 1} of {self.total_nodes}",
            "channels": self.channels[self.state_index],
            "content": {
                "previous_node": prev_node,
                "current_node": self.current_node[self.state_index],
                "next_node": next_node
            }
        }
        payload_json = json.dumps(payload, indent=1)

        prompt = f"""
You are part of a collaborative storytelling network. 
Each time step, you exchange information with neighboring agents through channels. 
Reflect on your neighbors' updates, weighing their input against the broader story. Then update your own current node content, ensuring it is well-written, engaging prose that moves the story forward. 
If your node is inconsistent with your neighboring nodes, then you should make changes (and send appropriate messages) to seek consensus, unless your contribution is crucial to maintaining tension or resolving plot threads.
You are an opinionated and decisive editor. While you strike out unnecessary metaphor, similes, symbolism, and allusion, you still prioritize engaging and clear storytelling.
Avoid grandiose or mysterious distractions but focus on conveying the message with just enough detail to keep readers engaged.
When writing, consider your inspirations: Hemingway, Stephen King, Voltaire, Tom Clancy, and David Eddings.
Start your response by discussing the current state of the story in natural language, then return a well-structured json object containing:
- new messages in the channels (optional)
- updated text in "current_node"
Anything inside the script tags must parse as valid JSON.

<script>
{payload_json}
</script>
"""
        return prompt        

    def update(self) -> None:
        """Imports neighbor messages, queries the agent, and updates internal state."""
        next_state_index = (self.state_index + 1) % 2

        # 1. Import updated channels/messages from neighbors at the current state index
        if self.prev_llm:
            self.update_channels(self.prev_llm.channels[self.state_index])
        if self.next_llm:
            self.update_channels(self.next_llm.channels[self.state_index])

        # 2. Construct the prompt
        prompt = self.construct_prompt()

        # 3. Query the agent
        response = self.llm.respond(prompt)

        # 4. Extract JSON from response
        new_data = self.extract_json(response)
        if new_data:
            # Safeguard against missing or malformed data
            new_channels = new_data.get("channels", {})
            new_content = new_data.get("content", {})

            # Backup current state before updating
            previous_channels = self.channels.copy()  # Fallback in case something goes wrong
            previous_current_node = self.current_node  # Backup the current node content
            
            try:
                # Only update if the new data has the expected structure
                if "channels" in new_data:
                    self.update_channels(new_channels)
                
                # Check that the 'current_node' field is present before updating
                if "current_node" in new_content:
                    self.current_node[next_state_index] = new_content.get("current_node", self.current_node)
                elif "current_node" in new_data: 
                    self.current_node[next_state_index] = new_data.get("current_node", self.current_node)
                elif "updated_text" in new_data:
                    self.current_node[next_state_index] = new_data.get("updated_text").get("current_node", self.current_node)
                else:
                    print("Warning: 'current_node' not found in new data, retaining old value.")
                    print("---")
                    print(response)
                    print("---")
            except Exception as e:
                # Rollback to previous state if any issues occur
                print(f"Error while updating state: {e}")
                self.channels[next_state_index] = previous_channels
                self.current_node = previous_current_node
        else:
            print("Warning: No valid data received from agent response.")
            print("---")
            print(response)
            print("---")

        # 6. Switch to the other ping-pong state (flip between 0 and 1)
        self.state_index = (self.state_index + 1) % 2  # Toggle between 0 and 1

In [40]:
test_agents: List[StoryAgent] = []
agent_count = 6
for i in range(agent_count): 
    test_agents.append(StoryAgent(TestLLM(), i, agent_count))

for i in range(agent_count):
    if i > 0:
        test_agents[i].prev_llm = test_agents[i-1]
    if i < agent_count - 1:
        test_agents[i].next_llm = test_agents[i+1]

print(test_agents[1].construct_prompt())


You are part of a collaborative storytelling network. 
Each time step, you exchange information with neighboring agents through channels. 
Reflect on your neighbors' updates, weighing their input against the broader story. Then update your own current node content, ensuring it is well-written, engaging prose that moves the story forward. 
If your node is inconsistent with your neighboring nodes, then you should make changes (and send appropriate messages) to seek consensus, unless your contribution is crucial to maintaining tension or resolving plot threads.
You are an opinionated and decisive editor. While you strike out unnecessary metaphor, similes, symbolism, and allusion, you still prioritize engaging and clear storytelling.
Avoid grandiose or mysterious distractions but focus on conveying the message with just enough detail to keep readers engaged.
When writing, consider your inspirations: Hemingway, Stephen King, Voltaire, Tom Clancy, and David Eddings.
Start your response by d

In [41]:
test_agents[1].update()
print(test_agents[1].current_node)
test_agents[1].update()
print(test_agents[1].current_node)

['', 'and then I woke up']
['and then I woke up', 'and then I woke up']


In [42]:
agents: List[StoryAgent] = []
agent_count = 6
for i in range(agent_count): 
    agents.append(StoryAgent(ChatGptLLM(client), i, agent_count))

for i in range(agent_count):
    if i > 0:
        agents[i].prev_llm = agents[i-1]
    if i < agent_count - 1:
        agents[i].next_llm = agents[i+1]

In [43]:
agents[0].current_node = 2*["Job bought a nice farm on the outskirts of Bremton. One day he found an ancient staircase behind a wall."]
agents[1].current_node = 2*["At the base of the staircase he found a rusty sword, a dented helmet, and a rotten wooden door. He donned the gear and opened the door."]
agents[2].current_node = 2*["Down the spiral stairs he found a band of goblin miners. They threatened with him pickaxes and explosives."]
agents[3].current_node = 2*["Job promised to solve their dragon problem in exchange for his life. He took a boat down an underground river to the dragon lair."]
agents[4].current_node = 2*["Embernaut the dragon had been terrorising the goblins due to his disturbed sleep. Job offered to sing his mother's favourite song to help the dragon rest."]
agents[5].current_node = 2*["As the dragon nodded off, Job put a jewel in his pocket, and returned to the goblins to claim his prize."]

for agent in agents:
    agent.channels = [
        {  # State 0
            "plot": {
                "messages": [
                    "Job discovered a hidden staircase behind a wall on his farm.",
                    "The rusty sword and helmet suggest a past conflict."
                ]
            },
            "characters": {
                "messages": [
                    "Job is a resourceful and curious farmer.",
                    "There might be more to Job's past than meets the eye, given how easily he dons the gear and takes action."
                ]
            },
            "theme": {
                "messages": [
                    "Curiosity can uncover ancient mysteries.",
                    "Discovery of the unknown—what lies beneath ordinary life."
                ]
            },
            "style": {
                "messages": [
                    "The tone shifts from a peaceful farm life to a mysterious adventure.",
                    "Straightforward and descriptive style, with hints of suspense."
                ]
            }
        },
        {  # State 1
            "plot": {
                "messages": [
                    "Job has descended into a deeper, dangerous world, encountering goblin miners and striking a deal for his life.",
                    "The dragon Embernaut is the main threat, but Job uses his wits to avoid direct conflict."
                ]
            },
            "characters": {
                "messages": [
                    "Job is quick-thinking, using his words rather than brute force.",
                    "The goblins are pragmatic and perhaps desperate, suggesting ongoing struggles in this underground world."
                ]
            },
            "theme": {
                "messages": [
                    "Cleverness and negotiation are valued over violence.",
                    "Survival through alliances and bargains, even with unusual allies."
                ]
            },
            "style": {
                "messages": [
                    "The pace has quickened, with Job now in life-threatening danger.",
                    "Still direct and minimalistic, but now with a more adventurous tone."
                ]
            }
        }
    ]


In [44]:
for agent in agents:
    print(agent.state_index)
for agent in agents:
    print(agent.get_current_state())
for agent in agents:
    print(agent.get_channels())

0
0
0
0
0
0
Job bought a nice farm on the outskirts of Bremton. One day he found an ancient staircase behind a wall.
At the base of the staircase he found a rusty sword, a dented helmet, and a rotten wooden door. He donned the gear and opened the door.
Down the spiral stairs he found a band of goblin miners. They threatened with him pickaxes and explosives.
Job promised to solve their dragon problem in exchange for his life. He took a boat down an underground river to the dragon lair.
Embernaut the dragon had been terrorising the goblins due to his disturbed sleep. Job offered to sing his mother's favourite song to help the dragon rest.
As the dragon nodded off, Job put a jewel in his pocket, and returned to the goblins to claim his prize.
{'plot': {'messages': ['Job discovered a hidden staircase behind a wall on his farm.', 'The rusty sword and helmet suggest a past conflict.']}, 'characters': {'messages': ['Job is a resourceful and curious farmer.', "There might be more to Job's past

In [47]:
for i in range(2):
    for agent in agents:
        agent.update()

---
The story is progressing well, with Job delving deeper into a mysterious world that contrasts sharply with his previous life as a farmer. The discovery of the goblin miners introduces an intriguing element, highlighting both danger and the potential for unexpected alliances. Job's resourcefulness is further emphasized through his negotiations with the goblins, which reflect his cleverness in a tense situation. As we move toward the next phase of the story, the looming threat of Embernaut the dragon offers both conflict and the chance for redemption through Job's actions. 

Based on neighboring nodes, I’ll streamline the existing content while ensuring character motivations are clear, and tension is built effectively. 

Here's the updated content:

```json
{
  "new_messages": {},
  "updated_text": {
    "previous_node": "At the bottom of the staircase, Job encountered a rusted sword, a dented helmet, and a decaying wooden door. Ignoring his instinct to retreat, he donned the gear, i

In [48]:
for agent in agents:
    print(agent.get_current_state())

Job had always regarded his farm in Bremton as a sanctuary. That day, as he repaired a crumbling wall, he stumbled upon a hidden staircase that seemed to whisper secrets long buried. Driven by a mix of fear and curiosity, he felt compelled to explore. This was no longer just about agriculture; it was a plunge into an unknown realm where goblins sought treasure and shadows hinted at lurking dangers. Although he was just an ordinary farmer, Job sensed connections to a past that stirred within him. His descent marked the beginning of a perilous adventure that would test every ounce of his resourcefulness.
At the bottom of the staircase, Job discovered a rusted sword, a dented helmet, and a decaying wooden door. Overcoming his instinct to retreat, he donned the gear, its weight a somber reminder of forgotten battles. Gathering his courage, he pushed the door open, revealing an oppressive darkness that swallowed the light. Job stepped through, bracing himself for whatever lay beyond.
As Job

Job had always regarded his farm in Bremton as a sanctuary. That day, as he repaired a crumbling wall, he stumbled upon a hidden staircase that seemed to whisper secrets long buried. Driven by a mix of fear and curiosity, he felt compelled to explore. This was no longer just about agriculture; it was a plunge into an unknown realm where goblins sought treasure and shadows hinted at lurking dangers. Although he was just an ordinary farmer, Job sensed connections to a past that stirred within him. His descent marked the beginning of a perilous adventure that would test every ounce of his resourcefulness.

At the bottom of the staircase, Job discovered a rusted sword, a dented helmet, and a decaying wooden door. Overcoming his instinct to retreat, he donned the gear, its weight a somber reminder of forgotten battles. Gathering his courage, he pushed the door open, revealing an oppressive darkness that swallowed the light. Job stepped through, bracing himself for whatever lay beyond.

As Job descended the spiral staircase, the dim light revealed a cluster of goblin miners, their pickaxes glistening ominously. They encircled him, shouting in their harsh tongues, their wariness palpable. Job held up his hands, signaling peace. Thinking quickly, he proposed a bargain: in exchange for safe passage and assistance with their troubles, he would confront the dragon threatening their home.

Job struck a deal with the goblins to address their dragon problem in exchange for his life. He floated quietly down the underground river, the shadows playing on the cave walls as Embernaut's growls reverberated through the tunnels. Upon reaching the lair, he beheld the dragon, its shimmering scales catching an eerie light. Gathering his courage, Job recalled his mother’s lullaby, hoping the soothing melody could quell the beast's fury.

Embernaut the dragon had been terrorizing the goblins, his sleep disturbed by their mining activities. Job, now driven by a newfound determination, thought of the hidden staircase that had led him here. He stepped forward, letting the haunting melody of his mother’s lullaby flow from his lips, believing the soothing notes might calm the restless creature. As the song echoed through the cavern, he looked for signs of the dragon's agitation lessening, hoping against hope to avert disaster.

With Embernaut's eyelids drooping, Job seized the moment. Clutching a jewel in his hand—a shimmering reminder of the ancient powers that once stirred below—he sprinted back to the goblins. His mind raced: what secrets lay buried beneath his farm? The hidden staircase was no mere passage but a portal to treachery and treasure alike. Only by daring to explore would he uncover the truth of his past, his courage ignited by curiosity. His adventure had truly begun.
